## ME447 Project 1
#### Amanda Schultz (amanda36) and Avanthi Obadage (obadage2)

### 2. 1/0 Knapsack Probelm 
#### Genetic Algorithm version

In [1]:
#initialize 
import numpy as np
from math import comb
A_data = np.load("A.npz") #dict format: n_items, capacity, item_values, item_weights
B_data = np.load("B.npz")

In [20]:
#Given
n_items = A_data["n_items"]
capacity = A_data["capacity"]
values = np.array(A_data["item_values"])
weights = np.array(A_data["item_weights"])


#### Representation 
##### Solutions are binary vectors; [1] is included and [0] is excluded

In [ ]:
# Representation
pop_size = 50 # how big is the population?
population = np.random.randint(2, size=(pop_size, n_items)) #some inital population with random objects

#Fitness
def fitness(individual):
    """
    A function that calculates the weight and value of an individual solution (parent) from the population
    """
    total_weight = np.sum(individual * weights)
    total_value = np.sum(individual * values)
    if total_weight > capacity:
        return 0  # overweight 
    return total_value

#Deterministic Variation Selection
num_parents = pop_size // 2 #next generation is half of previous
def select_parents(population, num_parents):
    """
    Selects the top solutions (parents) with highest fitness.
    """
    population = np.array(population)  # ensure it's an array
    fitness_vals = np.array([fitness(ind) for ind in population])
    top_idx = np.argsort(fitness_vals)[-num_parents:]  # largest fitness
    top_idx = top_idx[::-1]  
    return population[top_idx]

#Recombination
def crossover(parents, pop_size):
    """
    Crosses best parents to make offspring (new solutions) with (hopefully) better fitness
    """
    n_items = parents.shape[1]
    offspring = []
    while len(offspring) < pop_size:
        for i in range(len(parents)//2):
            p1 = parents[i]
            p2 = parents[-(i+1)]
            point = np.random.randint(1, n_items)
            c1 = np.concatenate([p1[:point], p2[point:]])
            c2 = np.concatenate([p2[:point], p1[point:]])
            offspring.append(c1)
            if len(offspring) < pop_size:
                offspring.append(c2)
    return np.array(offspring[:pop_size])

#Mutation
mutation_prob = 0.05 #some probability that a bit will flip 
def mutate(offspring, mutation_prob):
    """
    Randomly decides when a bit will flip to escape any local minima
    """
    mutation = np.random.rand(*offspring.shape) < mutation_prob
    return np.logical_xor(offspring, mutation).astype(int)

#Environmental Selection
def environmental_selection(offspring, pop_size):
    """
    Selects the top offspring individuals to form the next generation.
    """
    offspring = np.array(offspring) 
    fitness_vals = np.array([fitness(ind) for ind in offspring])
    top_idx = np.argsort(fitness_vals)[-pop_size:]  # largest fitness
    return offspring[top_idx]


# Run simulation!
generations = 50

for gen in range(generations):
    parents = select_parents(population, num_parents)
    
    # 4. Crossover
    offspring = crossover(parents, pop_size * 2)  # create extra to select from
    
    # 5. Mutation
    offspring = mutate(offspring, mutation_prob)
    
    # 6. Environmental selection (elitist)
    population = environmental_selection(offspring, pop_size)

# Solutions
fitness_vals = np.array([fitness(ind) for ind in population])
best_idx = np.argmax(fitness_vals)
best_solution = population[best_idx]
best_value = fitness_vals[best_idx]

print("Best solution:", best_solution)
print("Total value:", best_value)


Best solution: [0 1 1 0 0 0 0 1 0 0 0 0 0 0 0 1 0 1 1 0 0 0 1 0 1 1 1 1 1 1 1 0 0 1 1 1 0
 0 0 1 0 0 0 0 0 1 0 1 1 0]
Total value: 0


### 3. Minima of the Parabola
#### Genetic Algorithm version

In [3]:
#Initialization
import numpy as np
SEED = 42
rng = np.random.default_rng(seed=SEED)

In [4]:
#Representation
pop_size = 30
population = rng.uniform(-10,10, size = pop_size)
num_parents = pop_size//2

def parabola_problem(individual):
    return 10*individual**2

In [ ]:
#Calculate Fitness
def fitness(pop, problem):
    return [problem(element) for element in population]

#Select Parents
def select_fittest(pop, pop_fitness, num_par):
    sorted_ind = np.argsort(pop_fitness)
    chosen_ind = sorted_ind[:num_par]
    chosen_parents = pop[chosen_ind]
    return chosen_parents

#Crossover to generate offspring, taking the average between two randomly selected parents from the set
def crossover(parents,num_par, num_off):
    offspring = np.empty(num_off)
    for n in range(num_off):
        parent1_idx, parent2_idx = rng.choice(np.arange(num_par), size = 2, replace = True)
        offspring[n] = (parents[parent1_idx] + parents[parent2_idx])/2
    return offspring

#Mutate offspring, using the average
def mutate(offspring, prob):
    #mutation options? generate a new random number altogether? add some random number less than one to chosen children?
    
    return


gen 0 fitness =  0.00022944675107733516
parents =  [-3.46925335e-05  3.42912103e-04  3.69358404e-04 -6.37027433e-04
 -7.93998830e-04 -1.02637297e-03 -1.02637297e-03 -1.20843883e-03
 -1.20843883e-03 -2.31592933e-03  2.37452221e-03  2.37452221e-03
  2.66530059e-03  2.75212685e-03 -2.82560894e-03]
offspring =  [-9.10185902e-04 -1.92599095e-03 -9.10185902e-04 -4.14345682e-04
  7.71844007e-04  2.37452221e-03 -4.32763364e-04 -1.92599095e-03
  1.35871716e-03  7.90261689e-04  2.51991140e-03  1.54109785e-04
 -2.57076913e-03 -1.17531093e-03 -9.73285461e-04 -9.86508612e-04
  2.56332453e-03  1.31530403e-03 -6.37027433e-04 -3.28507285e-04
  2.51991140e-03  1.51732950e-03  1.35871716e-03 -1.55496408e-03
  1.35871716e-03 -3.46925335e-05  1.67332935e-04  1.74685634e-04
 -8.01541702e-05 -1.73131818e-03  9.35650882e-04 -3.35859983e-04
  2.51991140e-03 -3.67410449e-05  1.35871716e-03  2.51991140e-03
  2.66530059e-03 -5.30532754e-04 -8.31700204e-04  3.42912103e-04
  5.83041688e-04  5.83041688e-04 -1.76218